# Attention-based Seq2Seq Neural Machine Translation

문장 번역(translation) 문제에 대해 좀 더 깊이 생각해 보겠습니다. 사람이 문장을 번역할 때에는 일반적으로 다음과 같은 과정을 거칩니다:
1. <b>문장 전체 이해</b>: 먼저 원문 전체를 읽고, 전달하고자 하는 의미와 개념을 이해합니다.
2. <b>순차적 번역 생성</b>: 그 후, 한 번에 한 단어씩 번역을 작성해 나갑니다.
3. <b>Attention</b>: 번역을 진행하는 동안, 필요할 때마다 원문을 다시 참조하여 ‘현재 번역하고자 하는 단어’와 가장 관련이 깊은 부분에 주의(attention)를 기울입니다.

지난 실습에서 구현한 seq2seq 모델 구조에서는 1번 단계를 Encoder구조를 이용하여, 2번 단계를 Decoder구조를 이용하여 수행합니다.

이번 실습의 목표는 3번 단계 <b>Attention 메커니즘</b>을 Seq2Seq 모델에 구현하는 것입니다.

In [ ]:
import math
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import spacy

from helpers import TranslationDataset, train_one_epoch, evaluate_one_epoch

이전 실습과 동일하게, 프랑스어(French) 문장을 입력 받아 영어(English) 문장을 출력하는 기계 번역 모델을 직접 구현하고 학습해 보겠습니다.

이번 실습자료는 아래 두 논문에서 제시한 모델 구조를 기반으로 합니다.
1.  Bahdanau et al., [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473) (2014)
    - Seq2Seq에 <b>Attention</b>을 처음으로 도입한 연구
2.  Luong et al., [Effective Approaches to Attention-based Neural Machine Translation](https://arxiv.org/pdf/1508.04025) (2015)
    - Attention 기법을 보다 효율적이고 체계적으로 개선한 연구
    - 이번 실습은 <b>Luong-style Attention</b>을 구현하는 데 중점을 둡니다.

## Dataset
번역 모델 학습을 위한 `TranslationDataset`은 각 샘플을 튜플 `(source_sentence, target_sentence)`형식으로 반환합니다.

In [ ]:
dataset = TranslationDataset(path_tsv = "/datasets/NLP/eng-fra.txt")

test_size = int(0.1 * len(dataset))
valid_size = int(0.1 * len(dataset))
train_size = len(dataset) - test_size - valid_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size, test_size],
    generator=torch.Generator().manual_seed(42)
)
print(f"Train size: {len(train_dataset)}, Val size: {len(val_dataset)}, Test size: {len(test_dataset)}\n")

for source_sentence, target_sentence in train_dataset:
    print("Example source sentence:", source_sentence)
    print("Example target sentence:", target_sentence)
    break

## spaCy tokenizers
프랑스어와 영어 문장 토큰화를 위해서는 `spaCy` 라이브러리에서 제공하는 언어별 tokenizer를 활용합니다

In [ ]:
spacy_fr = spacy.load('fr_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')
tokenizer_french = get_tokenizer(lambda text: [tok.text.lower() for tok in spacy_fr.tokenizer(text)])
tokenizer_english = get_tokenizer(lambda text: [tok.text.lower() for tok in spacy_en.tokenizer(text)])

## Vocabulary
Tokenizer로 분할된 토큰들을 바탕으로 각 언어별 `vocabulary`를 생성합니다.

특수 토큰(special tokens)으로는 `<pad>`, `<unk>`, `<sos>`, `<eos>` 4가지를 사용합니다.

In [ ]:
PAD_TOKEN = "<pad>"  # Padding Token
UNK_TOKEN = "<unk>"  # Unknown Token (Out of Vocabulary)
SOS_TOKEN = "<sos>"  # Start of Sentence
EOS_TOKEN = "<eos>"  # End of Sentence

SPECIAL_TOKENS = [PAD_TOKEN, UNK_TOKEN, SOS_TOKEN, EOS_TOKEN]

source_token_list = [tokenizer_french(source) for source, _ in train_dataset]
target_token_list = [tokenizer_english(target) for _, target in train_dataset]

source_vocab = build_vocab_from_iterator(source_token_list, specials=SPECIAL_TOKENS, min_freq = 2)
target_vocab = build_vocab_from_iterator(target_token_list, specials=SPECIAL_TOKENS, min_freq = 2)

source_vocab.set_default_index(source_vocab[UNK_TOKEN])
target_vocab.set_default_index(target_vocab[UNK_TOKEN])

print(f"Vocab sizes → French: {len(source_vocab)}, English: {len(target_vocab)}")

print("Source vocab (French): ", source_vocab.get_itos()[:10])
print("Target vocab (English): ", target_vocab.get_itos()[:10])

PAD_TOKEN_IDX  = source_vocab[PAD_TOKEN]
SOS_TOKEN_IDX  = source_vocab[SOS_TOKEN]
EOS_TOKEN_IDX  = source_vocab[EOS_TOKEN]
assert PAD_TOKEN_IDX == target_vocab[PAD_TOKEN]
assert SOS_TOKEN_IDX == target_vocab[SOS_TOKEN]
assert EOS_TOKEN_IDX == target_vocab[EOS_TOKEN]

## Collate Function
텍스트 전처리 파이프라인은 다음과 같은 과정으로 구성됩니다.:
- <b>Tokenization</b> : 각 문장을 해당 언어의 tokenizer를 이용해 토큰 단위로 분할합니다.
- <b>Vocabulary 매핑</b>: 각 토큰을 해당 언어의 `Vocabulary`를 이용하여 정수 인덱스(token indices)로 변환합니다
- 문장의 시작에는 `<sos>`토큰을, 문장의 끝에는 `<eos>`토큰을 추가해줍니다.
- <b>Padding</b>: mini-batch내의 <b>가장 긴 시퀀스 길이</b>에 맞춰 padding을 수행합니다. 

In [ ]:
def collate_seq2seq_batch(batch):
    source_batch, target_batch = [], []
    for source_sentence, target_sentence in batch:
        source_indices = [SOS_TOKEN_IDX] + [source_vocab[t] for t in tokenizer_french(source_sentence)] + [EOS_TOKEN_IDX]
        target_indices = [SOS_TOKEN_IDX] + [target_vocab[t] for t in tokenizer_english(target_sentence)] + [EOS_TOKEN_IDX]
        source_batch.append(torch.tensor(source_indices, dtype=torch.long))
        target_batch.append(torch.tensor(target_indices, dtype=torch.long))

    # pad to max_length in batch
    source_batch = torch.nn.utils.rnn.pad_sequence(source_batch, padding_value=PAD_TOKEN_IDX, batch_first=True)
    target_batch = torch.nn.utils.rnn.pad_sequence(target_batch, padding_value=PAD_TOKEN_IDX, batch_first=True)
    return source_batch, target_batch



batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_seq2seq_batch)
valid_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_seq2seq_batch)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_seq2seq_batch)

for X, y in test_loader:
    print("Mini-batch X shape:", X.shape)
    print("Mini-batch y shape:", y.shape)
    print("\n1st source text:", X[0])
    print("1st target text:", y[0])
    break

print("\n<sos> token index:", SOS_TOKEN_IDX)
print("<eos> token index:", EOS_TOKEN_IDX)
print("<pad> token index:", PAD_TOKEN_IDX)

## Building the Attention-based Seq2Seq Model

### Encoder

인코더(Encoder)는 번역할 문장(source sentence)을 입력 받아 순환 신경망(RNN, LSTM, GRU)을 통해 각 token을 순차적으로 인코딩합니다.

입력 시퀀스 $X = (x_1, x_2, \dots, x_{T_x})$가 주어졌을때, Encoder GRU는 각 time-step $t$에서 아래 수식에 따라 hidden state를 업데이트합니다.

$$
\mathbf{h}_t = \text{GRU}_{\text{enc}}(x_t, \mathbf{h}_{t-1})
$$

 - $\mathbf{h}_t$는 현재까지 입력된 토큰들 $x_{1:t}$에 대한 정보를 압축하여 담고 있는 상태 벡터입니다.

<mark>실습</mark> `Encoder` 모듈을 완성하세요
1. Embedding layer: `self.embedding`레이어를 이용하여 입력 토큰 인덱스를 임베딩 벡터로 변환합니다.
2. Droptout layer: 과적합 방지를 위해 임베딩 벡터에 `self.dropout`레이어를 적용합니다.
3. GRU layer: 드롭아웃이 적용된 임베딩 벡터를 `self.gru`레이어에 입력합니다.
   - GRU 레이어는 다음과 같은 두 가지 텐서를 반환합니다 ([docs](https://docs.pytorch.org/docs/stable/generated/torch.nn.GRU.html)):
     - `output` : 마지막 GRU layer의 <b>모든 time-step</b>에서의 hidden state $(h_1, h_2, \dots, h_{T_x})$
       - shape = `(batch_size, src_len, hidden_dim)`
     - `hidden` : 모든 GRU layer의 <b>final hidden state</b> $h_{T_x}$.
       - shape = `(num_layers, batch_size, hidden_dim)`

4. 최종적으로 인코더는 `output`과 `hidden`을 반환합니다.
   - `output`(모든 time-step에서의 hidden state) : 어텐션(attention)의 입력으로 사용됩니다.
   - `hidden`(final hidden state) : 디코더의 초기 hidden state로 사용됩니다.

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers, pad_token_index, dropout=0.1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx = pad_token_index)
        self.gru = nn.GRU(embed_dim, hidden_dim, num_layers = num_layers,
                          batch_first = True, dropout = dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        """
        Args:
            src: Tensor of shape (batch_size, src_len) containing token indices for the source sentence.
        Returns:
            output: Tensor of shape (batch_size, src_len, hidden_dim)
                    containing the hidden states from the last GRU layer for all time steps.
            hidden: Tensor of shape (num_layers, batch_size, hidden_dim)
                    containing the final hidden states of all GRU layers.

        """
        ##### YOUR CODE START #####


        ##### YOUR CODE END #####

        return output, hidden  # `output` for attention, `hidden` for decoder hidden state initialization

In [ ]:
encoder = Encoder(vocab_size = len(source_vocab), embed_dim = 128, hidden_dim = 256,
                  num_layers = 2, pad_token_index = PAD_TOKEN_IDX, dropout = 0.5)

X = torch.randint(0, 100, (8, 20))  # (batch_size, src_len)
output, hidden = encoder(X)

print("Input shape:", X.shape)                          # (batch_size, src_len)
print("Encoder output shape:", output.shape)            # (batch_size, src_len, hidden_dim)
print("Encoder hidden state shape:", hidden.shape)      # (num_layers, batch_size, hidden_dim)

assert output.shape == (8, 20, 256), f"Expected output shape (8, 20, 256), but got {hidden.shape}"
assert hidden.shape == (2, 8, 256), f"Expected hidden shape (2, 8, 256), but got {hidden.shape}"

### Dot-product similarity
두 벡터간의 dot-product similarity를 계산하기 위해서는 `torch.dot`연산을 사용합니다.

In [ ]:
vec1 = torch.rand(size = (4,))  # shape: (hidden_dim,)
vec2 = torch.rand(size = (4,))  # shape: (hidden_dim,)

similarity = torch.dot(vec1, vec2)

print(similarity.item())

벡터(하나의 token)와 행렬(sequence of token)간의 dot-product similarity를 계산하기 위해서는 for loop를 사용할 수 있습니다.

In [ ]:
query = torch.rand(size = (4,))   # shape: (hidden_dim,)
keys = torch.rand(size = (3, 4))  # shape: (seq_len, hidden_dim)

scores = torch.empty(size = (keys.shape[0],))
for i in range(keys.shape[0]):
    scores[i] = (torch.dot(query, keys[i]))
    
print(scores)  # shape: (seq_len,)

위의 반복문은 행렬 곱으로 한 번에 처리할 수 있습니다 (vectorization)

In [ ]:
# (seq_len, hidden_dim) @ (hidden_dim,) = (seq_len,)
scores_vec = keys @ query # (3, 4) @ (4,) -> (3,)

print(scores_vec)

mini-batch로 구성된 벡터와 행렬 간의 dot-product 연산을 수행하기 위해서는 `torch.bmm`([docs](https://docs.pytorch.org/docs/stable/generated/torch.bmm.html)) 함수를 사용합니다.
- `torch.bmm`은 batch matrix multiplication의 약자로 배치간의 행렬 곱을 수행합니다.
- `torch.bmm`의 입력은 반드시 3차원 텐서 `(b, n, m)` 형태여야 하므로, `unsqueeze` ([docs](https://docs.pytorch.org/docs/stable/generated/torch.unsqueeze.html))와 `squeeze` ([docs](https://docs.pytorch.org/docs/stable/generated/torch.squeeze.html))함수를 사용하여 차원을 적절히 맞춰줍니다.

In [ ]:
query = torch.randn(2, 4)     # (batch_size, hidden_dim)
keys = torch.randn(2, 3, 4)   # (batch_size, seq_len, hidden_dim)

# query를 (batch_size, hidden_dim) → (batch_size, hidden_dim, 1)로 변환
query = query.unsqueeze(2)    # (2, 4, 1)

# (batch_size, seq_len, hidden_dim) @ (batch_size, hidden_dim, 1) = (batch_size, seq_len, 1)
scores_bmm = torch.bmm(keys, query)   # (2, 3, 4) @ (2, 4, 1) = (2, 3, 1)
scores_bmm = scores_bmm.squeeze(2)    # (2, 3)

print(scores_bmm.shape)  # (batch_size, seq_len)

### Dot-product Attention

Attention mechanism은 디코더가 번역 과정의 각 시점에서 입력 시퀀스의 어느 부분에 주목(attention)해야하는지를 학습하는 방법입니다.

1. 인코더(Encoder)는 입력 시퀀스 $X = (x_1, x_2, \dots, x_{T_x})$ 에 대해 다음과 같은 hidden states를 생성합니다:

$$\mathbf{h}_1, \mathbf{h}_2, \dots, \mathbf{h}_{T_x} \in \mathbb{R}^{d_h}$$

2. 디코더(decoder) hidden state update:
   - time-step $t$에서 디코더는 이전 상태 $\mathbf{s}_{t-1}$와 이전 출력 토큰 $\mathbf{y}_{t-1}$을 기반으로 현재 상태를 업데이트합니다.
$$\mathbf{s}_t = \text{GRU}_{dec}(\mathbf{y}_{t-1}, \mathbf{s}_{t-1})$$

3. Attention score 계산:
   - 디코더 hidden state $\mathbf{s}_t$와 각 인코더 hidden state $\mathbf{h}_i$와 사이의 유사도를 계산합니다:

    $$
    e_{t,i} = f_{attn}(\mathbf{s}_t, \mathbf{h}_i) \in \mathbb{R}, \quad \mathbf{e}_t = [e_{t,1}, e_{t,2}, \dots, e_{t,T_x}] \in \mathbb{R}^{T_x}
    $$

   - 이번 실습에서는 Weighted dot-product attention를 사용하여 어텐션 스코어를 계산합니다.

        $$e_{t,i} = f_{attn}(\mathbf{s}_t, \mathbf{h}_i) = \mathbf{s}_t^\top \mathbf{W}_K \mathbf{h}_i$$

       - 여기서 $\mathbf{W}_K$는 학습가능한 가중치 행렬입니다.
       - 내적(dot product)은 두 벡터가 같은 방향일수록 큰 값을 가지므로, 두 벡터간 유사성을 측정하는 자연스러운 방법입니다.


4. Attention weights (Softmax normalization)
   - 스코어 벡터 $\mathbf{e}_t$를 소프트맥스로 정규화하여 확률 분포 형태의 어텐션 가중치를 얻습니다:

    $$
    \alpha_{t,i} = \frac{\exp(e_{t,i})}{\sum_{j=1}^{T_x} \exp(e_{t,j})} \in \mathbb{R},
    \quad
    \boldsymbol{\alpha}_t = \text{softmax}(\mathbf{e}_t) \in [0,1]^{T_x}
    $$

    $$\sum_{i=1}^{T} \alpha_{t,i} = 1$$

5. Context Vector (Attention output) 계산
    - Attention weights를 이용하여 인코더 hidden state들의 가중합(weighted sum)을 계산합니다

        $$ \mathbf{c}_t = \sum_{i=1}^{T_x} \alpha_{t,i} \mathbf{h}_i \in \mathbb{R}^{d_h}$$

        - 계산된 Context vector $\mathbf{c}_t$는 현재 번역 시점에서 필요한 입력 문장의 핵심 정보들을 담고있습니다.

---

<mark>실습</mark> `DotProductAttention` 모듈을 완성하세요
1. `query`: Decoder hidden state 중 마지막 GRU 레이어의 hidden state를 query vector로 사용합니다.
   - Shape: `(batch_size, hidden_dim)`
2. `keys` : `encoder_outputs`에 선형 변환 (`self.key_projection`)을 적용하여 key vector들을 얻습니다.
   - Shape: `(batch_size, src_len, hidden_dim)`
3. `values` : `encoder_outputs`을 그대로 value vector로 사용합니다
   - Shape: `(batch_size, src_len, hidden_dim)`
4. `attention_scores` : `query`와 `keys`의 내적을 구한 뒤, `self.scale`를 곱해줍니다.
   - `torch.bmm`, `unsqueeze`, `squeeze`를 이용하세요.
   - `self.scale`를 곱하여 softmax를 적용하기 전 텐서의 스케일을 조정해줍니다.
   - 결과 텐서 Shape: `(batch_size, src_len)`
5. `attention_weights` : `attention_scores`에 `torch.softmax`를 적용합니다.
6. `context_vector` : 계산된 `attention_weights`를 이용해 `values`의 weighted sum을 계산합니다
   - `torch.bmm`, `unsqueeze`, `squeeze`를 이용하세요.
   - 결과 텐서 Shape: `(batch_size, hidden_dim)`

In [ ]:
class DotProductAttention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.key_projection = nn.Linear(hidden_dim, hidden_dim, bias = False)
        self.scale = 1.0 / math.sqrt(hidden_dim)

    def forward(self, decoder_hidden, encoder_outputs):
        """
        Args:
            decoder_hidden: Tensor of shape (num_layers, batch_size, hidden_dim)
                            containing the decoder's current hidden state.
            encoder_outputs: Tensor of shape (batch_size, src_len, hidden_dim)
                             containing the encoder's outputs for all time step.
        Returns:
            context_vector: Tensor of shape (batch_size, hidden_dim)
                            containing weighted sum of encoder_outputs.
            attention_weights: Tensor of shape (batch_size, src_len)
                               containing attention weights over source tokens.
        """

        query = ...  # TODO. Shape: (batch_size, hidden_dim)
        keys = ... # TODO. Shape: (batch_size, src_len, hidden_dim)
        values = ... # TODO. Shape: (batch_size, src_len, hidden_dim)
          
        attention_scores = ... # TODO. Shape: (batch_size, src_len)
        attention_weights = ... # TODO. Shape: (batch_size, src_len)

        context_vector = ... # TODO. Shape: (batch_size, hidden_dim)

        return context_vector, attention_weights

In [ ]:
attention_layer = DotProductAttention(hidden_dim = 256)
decoder_hidden = torch.randn(2, 8, 256)      # (num_layers, batch_size, hidden_dim)
encoder_outputs = torch.randn(8, 20, 256)    # (batch_size, src_len, hidden_dim)
context_vector, attention_weights = attention_layer(decoder_hidden, encoder_outputs)

print("context_vector shape:", context_vector.shape)        # (batch_size, hidden_dim)
print("attention_weights shape:", attention_weights.shape)  # (batch_size, src_len)
assert context_vector.shape == (8, 256), f"Expected context_vector shape (8, 256), but got {context_vector.shape}"
assert attention_weights.shape == (8, 20), f"Expected attention_weights shape (8, 20), but got {attention_weights.shape}"

### Decoder

디코더(Decoder)에서는 어텐션(attention)을 활용하여 인코더 출력으로 부터 `context vector`를 계산하고, 타겟 문장을 한 단어씩 순차적으로 생성합니다

<mark>실습</mark> `Decoder` 모듈을 완성하세요

 - `Decoder`는 한번에 한 time-step의 디코딩만 수행합니다. 즉 입력 토큰의 길이(sequence length)는 항상 `1`입니다. 
 - `seq_length = 1`이므로, 텐서의 차원 처리에 유의하세요.
   - 필요에 따라 [`unsqueeze`](https://docs.pytorch.org/docs/main/generated/torch.unsqueeze.html)와 [`squeeze`](https://docs.pytorch.org/docs/main/generated/torch.squeeze.html)메서드를 활용하여 텐서의 차원을 적절히 변환하세요.

계산 과정
1. Embedding & Dropout: 입력 토큰 인덱스를 `self.embedding`레이어를 통해 임베딩 벡터로 변환합니다. 과적합 방지를 위해 임베딩 벡터에 `self.dropout`를 적용합니다.
2. Decoder GRU: dropout을 통과한 임베딩 벡터와 이전 hidden states $\mathbf{s}_{t-1}$를 `self.gru`에 입력하여 새로운 hidden state $\mathbf{s}_{t}$를 얻습니다.
3. Attention : `self.attention`를 이용하여 `context_vector` $\mathbf{c}_t$와 `attention_weights` $\mathbf{\alpha}_t$를 계산합니다.
4. fully-connected layer: `self.fc_out`(`nn.Linear`) 레이어를 이용하여 다음 단어의 예측값(`logits`)을 계산합니다.
   - $[\mathbf{s}_t; \mathbf{c}_t]$ ($[;]$는 concatenation)를 `self.fc_out`의 입력으로 전달합니다.
     - 마지막 GRU layer의 hidden state를 $\mathbf{s}_t$로 사용하세요.
   - `torch.cat`([docs](https://docs.pytorch.org/docs/stable/generated/torch.cat.html))을 이용하여 두 텐서를 concat합니다.
   - <mark>주의</mark> concat 순서는 일반적으로 어떤 순서로 하여도 무방하나 이번 실습에서는 채점을 위해 $[\mathbf{s}_t; \mathbf{c}_t]$의 <b>순서를 반드시 지켜주세요</b>.

In [ ]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers, pad_token_index, dropout=0.1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx = pad_token_index)
        self.gru = nn.GRU(embed_dim, hidden_dim, num_layers = num_layers,
                          batch_first = True, dropout = dropout)
        
        self.attention = DotProductAttention(hidden_dim)

        self.fc_out = ... # TODO
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_token, prev_hidden, encoder_outputs):
        """
        Args:
            input_token: Tensor of shape (batch_size,) containing token index for the current time step.
            prev_hidden: Tensor of shape (num_layers, batch_size, hidden_dim)
                         containing the previous hidden state of the decoder (or encoder's final hidden state).
            encoder_outputs: Tensor of shape (batch_size, src_len, hidden_dim)
                             containing the encoder outputs for attention.
        Returns:
            logits: Tensor of shape (batch_size, target_vocab_size) containing prediction scores for the next token.
            decoder_hidden : Tensor of shape (num_layers, batch_size, hidden_dim)
                             containing the updated decoder hidden state.
            attention_weights : Tensor of shape (batch_size, src_len) containing attention weights.
        """

        ## 1. Embed input token
        input_token = input_token.unsqueeze(1)  # Shape : (batch_size, 1)
        embedded_token = self.dropout(self.embedding(input_token))  # Shape : (batch_size, 1, embed_dim)

        ##### YOUR CODE START #####
        ## 2. Update decoder hidden state
        
        
        ## 3. Compute attention weights and context vector
        

        ## 4. Concatenate decoder hidden state and context vector, then generate output logits


        ##### YOUR CODE END #####

        return logits, decoder_hidden, attention_weights

In [ ]:
dec = Decoder(vocab_size = len(target_vocab), embed_dim = 128, hidden_dim = 256,
              num_layers = 2, pad_token_index = PAD_TOKEN_IDX, dropout = 0.5)

input_token = torch.randint(0, 100, (8,))  # (batch_size,)
prev_hidden = torch.randn(2, 8, 256)       # (num_layers, batch_size, hidden_dim)
encoder_outputs = torch.randn(8, 20, 256)  # (batch_size, src_len, hidden_dim)

logits, decoder_hidden, attention_weights = dec(input_token, prev_hidden, encoder_outputs)

print("Decoder logits shape:", logits.shape)               # (batch_size, target_vocab_size)
print("Decoder hidden state shape:", decoder_hidden.shape) # (num_layers, batch_size, hidden_dim)
print("attention_weights shape:", attention_weights.shape) # (batch_size, src_len)

assert logits.shape == (8, len(target_vocab)), f"Expected logits shape (8, {len(target_vocab)}), but got {logits.shape}"
assert hidden.shape == (2, 8, 256), f"Expected hidden shape (2, 8, 256), but got {hidden.shape}"
assert attention_weights.shape == (8, 20), f"Expected attention_weights shape (2, 20), but got {attention_weights.shape}"

### Seq2Seq

앞서 구현한 `Encoder`와 `Decoder`를 결합하여 `Seq2Seq` 모델을 완성합니다.

`Seq2Seq` 모델 구조는 지난 실습과 동일합니다

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.target_vocab_size = self.decoder.embedding.num_embeddings

    def forward(self, src, tgt, teacher_forcing_prob = 0.5):
        """
        Args:
            src: Tensor of shape (batch_size, source_seq_len) containing token indices for the source sentences.
            tgt: Tensor of shape (batch_size, target_seq_len) containing token indices for the target sentences.
            teacher_forcing_prob: Probability of using the ground-truth token as the next input.

        Returns:
            output_logits: Tensor of shape (batch_size, target_seq_len, target_vocab_size)
        """

        batch_size, target_seq_len = tgt.size()
        output_logits = torch.zeros(batch_size, target_seq_len, self.target_vocab_size, device=tgt.device)

        # Encode source sequence
        encoder_outputs, hidden = self.encoder(src)

        # First decoder input is <sos>
        current_input_token = tgt[:,0]  # (batch_size,)

        # Decode one step at at time
        for t in range(1, target_seq_len):
            decoder_logits, hidden, _ = self.decoder(current_input_token, hidden, encoder_outputs)
            output_logits[:, t] = decoder_logits

            # choose next input token: teacher forcing or model’s own prediction
            use_teacher_forcing = torch.rand(1).item() < teacher_forcing_prob
            current_input_token = tgt[:, t] if use_teacher_forcing else decoder_logits.argmax(1)
        return output_logits

In [ ]:
embed_dim = 256
hidden_dim = 512
num_layers = 2

enc = Encoder(vocab_size = len(source_vocab), embed_dim = embed_dim, hidden_dim = hidden_dim,
              num_layers = num_layers, pad_token_index = PAD_TOKEN_IDX, dropout = 0.5)

dec = Decoder(vocab_size = len(target_vocab), embed_dim = embed_dim, hidden_dim = hidden_dim,
              num_layers = num_layers, pad_token_index = PAD_TOKEN_IDX, dropout = 0.5)

model = Seq2Seq(enc, dec)


X = torch.randint(0, 100, (32, 20)) # (batch_size, src_len)
y = torch.randint(0, 100, (32, 30)) # (batch_size, tgt_len)

logits = model(X, y, teacher_forcing_prob=0.5)
print("Seq2Seq logits shape:", logits.shape)  # (batch_size, tgt_len, target_vocab_size)

assert logits.shape == (32, 30, len(target_vocab)), f"Expected logits shape (32, 30, {len(target_vocab)}), but got {logits.shape}"

<mark>실습</mark> 완성한 Attention-based Seq2Seq 번역 모델을 학습해보세요.
 - 학습에는 <u>약 1~2분</u> 정도 소요됩니다.

In [ ]:
def train_seq2seq():

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    SOURCE_VOCAB_SIZE   = len(source_vocab)
    TARGET_VOCAB_SIZE  = len(target_vocab)
    embed_dim       = 256
    hidden_dim      = 512
    num_layers      = 1
    encoder_dropout = 0.1
    decoder_dropout = 0.1
    grad_clip        = 1.0

    num_epochs = 3
    batch_size = 64
    learning_rate = 1e-3

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, 
                              shuffle=True, num_workers= 4, collate_fn=collate_seq2seq_batch)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, 
                              shuffle=False, num_workers= 4, collate_fn=collate_seq2seq_batch)
    test_dataloader  = DataLoader(test_dataset, batch_size=batch_size, 
                              shuffle=False, num_workers= 4, collate_fn=collate_seq2seq_batch)

    enc = Encoder(vocab_size = SOURCE_VOCAB_SIZE, embed_dim = embed_dim, hidden_dim = hidden_dim, 
                  num_layers = num_layers, pad_token_index = PAD_TOKEN_IDX, dropout = encoder_dropout)
    dec = Decoder(vocab_size = TARGET_VOCAB_SIZE, embed_dim = embed_dim, hidden_dim = hidden_dim, 
                  num_layers = num_layers, pad_token_index = PAD_TOKEN_IDX, dropout = decoder_dropout)
    model = Seq2Seq(enc, dec).to(device)


    print(f'The model has {sum(p.numel() for p in model.parameters() if p.requires_grad)} trainable parameters')

    criterion = nn.CrossEntropyLoss(ignore_index = PAD_TOKEN_IDX)
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

    for epoch in range(num_epochs):
        train_loss = train_one_epoch(model, device, train_dataloader, criterion, optimizer, epoch, grad_clip)
        val_loss = evaluate_one_epoch(model, device, val_dataloader, criterion, epoch)

        print(f"Epoch {epoch + 1:02} | Train Loss: {train_loss:.3f} | Val. Loss: {val_loss:.3f}")
        print(f'\t | Train PPL: {math.exp(train_loss):7.3f} | Val. PPL: {math.exp(val_loss):7.3f}')

    test_loss = evaluate_one_epoch(model, device, test_dataloader, criterion)
    print(f"| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |")

    return model

In [ ]:
model = train_seq2seq()

코드 구현이 잘 되었다면 별도의 하이퍼파라미터 튜닝(hyperparameter tuning)없이 `Validation PPL < 8.2`를 달성하실 수 있습니다

---

## Attention Visualization
어텐션(attention) 메커니즘의 가장 큰 장점 중 하나는 출력을 생성할 때 입력 문장의 어떤 부분을 참고했는지 시각화할 수 있다는 점입니다.

이는 단순히 모델의 성능을 개선하는 것을 넘어, <b>모델의 해석 가능성(interpretability)</b>을 크게 향상시킵니다.

In [ ]:
def translate_sentence(sentence, model, max_len = 50):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model.eval()

    tokens = [SOS_TOKEN] + tokenizer_french(sentence) + [EOS_TOKEN]
    source_indices = torch.tensor(source_vocab(tokens), dtype=torch.long).unsqueeze(0).to(device)

    with torch.no_grad():
        enc_outputs, hidden = model.encoder(source_indices)

        input_tok = torch.tensor([SOS_TOKEN_IDX], device=device)
        result_tokens = []

        assert enc_outputs.shape[1] == len(tokens)

        attentions = torch.zeros(max_len, len(tokens))
        for i in range(max_len):
            pred, hidden, attn = model.decoder(input_tok, hidden, enc_outputs)
            attentions[i] = attn

            next_token_idx = pred.argmax(1).item()
            result_tokens.append(target_vocab.lookup_token(next_token_idx))
            input_tok = torch.tensor([next_token_idx], device=device)

            if next_token_idx == EOS_TOKEN_IDX:
                break
    return tokens, result_tokens, attentions[:len(result_tokens)]

# Quick sanity check
example_fr = "Je suis désolé si je vous ai embarrassés."
print(f"FR ➡️ EN  : {example_fr}")

src_tokens, translation, attention = translate_sentence(example_fr, model)
print(f"Predicted: {translation}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
def plot_attention(sentence, translation, attention):
    fig, ax = plt.subplots(figsize=(10, 10))
    cax = ax.matshow(attention, cmap="bone")
    # fig.colorbar(cax)

    ax.set_xticks(ticks=np.arange(len(sentence)), labels=sentence, rotation=90, size=15)
    ax.set_yticks(ticks=np.arange(len(translation)), labels=translation, size=15)

    plt.tight_layout()
    plt.show()
    plt.close()

plot_attention(src_tokens, translation, attention)

위 어텐션 시각화를 살펴보면 각 출력 토큰을 생성할때 입력의 적절한 부분에 주의를 집중하고 있다는 점을 확인할 수 있습니다
 - `Je` <-> `I`
 - `désolé` <-> `sorry`
 - `si` <-> `if`
 - `embarrassés` <-> `embarassed`
 - `.` <-> `<eos>`